# Shopee Product Classification 

Notebook to experiment over several Neural Networks over the product dataset obtained from Shopee and evaluate results.

The following models are evaluated as part of this notebook:

Baseline 1: CNN <br>
Baseline 2: CNN with augmented layers <br>
Improvement 1: Adding ANN <br>
Improvement 2: Adding RNN <br>

## Imports and Config

In [1]:
!pip install scikit-image
!pip install shopee_crawler

You should consider upgrading via the 'c:\users\shubh\.pyenv\pyenv-win\versions\3.7.9\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\shubh\.pyenv\pyenv-win\versions\3.7.9\python.exe -m pip install --upgrade pip' command.


In [1]:
import os
import json

import numpy as np
import pandas as pd

from utils import file_utils
from scripts import crawler

## Data Load

In [2]:
image_dir = 'data/images'

In [3]:
if(file_utils.check_images_dir()):
    print(f'Images already exist at: {file_utils.images}')
elif(file_utils.check_tar_exists()):
    print(f'Images not yet extracted')
    print(f'Image zip exists at: {file_utils.images_zip}')
    file_utils.extract_tar()
else:
    print(f'Images need to be downloaded')
    for c in category_urls:
        get_category_data('data', c)
    download_images('data')

Images already exist at: data/images


In [4]:
images = file_utils.load_images_skimage(image_dir)

  6%|██████▍                                                                                            | 5079/78675 [01:01<25:47, 47.55it/s]

KeyboardInterrupt: 

['.DS_Store',
 '._.DS_Store',
 '._Home-Appliances-cat',
 '._Pet-Food-Supplies-cat',
 '._Watches-cat',
 'Automotive-cat',
 'Beauty-Personal-Care-cat',
 'Cameras-Drones-cat',
 'Computers-Peripherals-cat',
 'Dining-Travel-Services-cat',
 'Food-Beverages-cat',
 'Health-Wellness-cat',
 'Hobbies-Books-cat',
 'Home-Appliances-cat',
 'Home-Living-cat',
 'Jewellery-Accessories-cat',
 'Kids-Fashion-cat',
 "Men's-Bags-cat",
 "Men's-Shoes-cat",
 "Men's-Wear-cat",
 'Miscellaneous-cat',
 'Mobile-Gadgets-cat',
 'Pet-Food-Supplies-cat',
 'ShopeePay-Near-Me-cat',
 'Sports-Outdoors-cat',
 'Toys-Kids-Babies-cat',
 'Travel-Luggage-cat',
 'Video-Games-cat',
 'Watches-cat',
 "Women's-Apparel",
 "Women's-Bags-cat",
 "Women's-Shoes-cat"]

In [6]:
#TODO: Add interactive image visualizer

In [7]:
#TODO: Check if need to normalize data or convert shapes